# Encoding comparison

This notebook contains a side-by-side evaluation of different bit-to-tree encodings

In [2]:
import sys
sys.path.append('../')
sys.setrecursionlimit(10000) # Croissant

import numpy as np
import math
import plotly.express as px
import pandas as pd
from func_timeout import func_timeout, FunctionTimedOut
import traceback

from tree_lib.tree import TreeNode
from tree_lib.encodings import two_choices, two_choices_one_two, counting_ones, counting_ones_leaves, fibonaccio, tuples, square, cube, entropy, power_digits, power_digits_2, gauss, cantor2d, simplex, one_child
import tree_lib.util as util



#### 1 - Define encodings to compare

In [3]:
ENCODINGS_UNFILTERED = [
    # ["gauss", gauss.bits_to_tree, gauss.tree_to_bits],
    ["two_choices", two_choices.bits_to_tree, two_choices.tree_to_bits],
    # ["two_choices_one_two", two_choices_one_two.bits_to_tree, two_choices_one_two.tree_to_bits],
    ["two_choices_wrap", two_choices.bits_to_tree_wrap, None],
    ["two_choices_mark", two_choices.bits_to_tree_mark, None],
    # ["two_choices_k=9", lambda s: two_choices.bits_to_tree_k(s, 9), None],
    # ["counting_ones_leaves", counting_ones_leaves.bits_to_tree, counting_ones_leaves.tree_to_bits],
    # ["fibonaccio", fibonaccio.bits_to_tree, fibonaccio.tree_to_bits],
    # ["power_digits", power_digits.bits_to_tree, power_digits.tree_to_bits],
    # ["power_digits_2", power_digits_2.bits_to_tree, power_digits_2.tree_to_bits],
    # ["tuples", tuples.bits_to_tree, tuples.tree_to_bits],
    ["square", square.bits_to_tree, square.tree_to_bits],
    # ["cube", cube.bits_to_tree, cube.tree_to_bits],
    # ["entropy", entropy.bits_to_tree, entropy.tree_to_bits],
    # ["cantor2d", cantor2d.bits_to_tree, cantor2d.tree_to_bits],
    ["simplex2d", lambda s: simplex.bits_to_tree(s, 2), lambda t: simplex.tree_to_bits(t, 2)],
    ["simplex3d", lambda s: simplex.bits_to_tree(s, 3), lambda t: simplex.tree_to_bits(t, 3)],
    ["one_child", one_child.bits_to_tree, one_child.tree_to_bits],
    # ["simplex4d", lambda s: simplex.bits_to_tree(s, 4), lambda t: simplex.tree_to_bits(t, 4)],
    # ["simplex5d", lambda s: simplex.bits_to_tree(s, 5), lambda t: simplex.tree_to_bits(t, 5)],
    # ["simplex10d", lambda s: simplex.bits_to_tree(s, 10), lambda t: simplex.tree_to_bits(t, 10)],
    ]

#### 2. Check for correctness

In [4]:

from func_timeout import func_timeout, FunctionTimedOut

BASIC_STRINGS = [
    "0","1","01","10","11", "100", "101", "110", "111",     # basic
    "010101010101", "0000000000", "1111111111",             # something more complex
]
LONG_STRINGS = [util.gen_bit_string(300) for _ in range(300)]

def string_test(strings, name, btt, ttb, shuffle=False):
    def encode_decode(s):
        t = btt(s)
        if t is not None and shuffle:
            util.shuffle_tree(t)
        return ttb(t)

    try:
        if ttb is None:
            # If there is no ttb, try to encode strings only
            res = [btt(s) for s in strings]
            return f"🤷️️️️️️"
        else:
            errors = func_timeout(10., lambda: [s for s in strings if encode_decode(s) != s])
            return "✅" if len(errors)==0 else f"🐖️️️️️️ {errors[0]}"
    except FunctionTimedOut:
        return f"⏰"
    except Exception as e:
        print(traceback.format_exc())
        return f"🧨️️️️️️ {e}"

results = [[name, 
            string_test(BASIC_STRINGS, name, btt, ttb), 
            string_test(LONG_STRINGS, name, btt, ttb),
            string_test(BASIC_STRINGS+LONG_STRINGS, name, btt, ttb, True),] 
            for name, btt, ttb in ENCODINGS_UNFILTERED]

df = pd.DataFrame(results, columns=['Name', 'Basic Test', "Long Strings", "Unordered"])

# Discard encodings that do not work
def is_acceptable_result(test_basic,test_long,test_unordered):
    return ("✅" in test_basic or "🤷️️️️️" in test_basic) and \
           ("✅" in test_long or "🤷️️️️️" in test_long)

to_discard = [name for [name, t1, t2, t3] in results if not is_acceptable_result(t1,t2,t3)]
if to_discard:
    print(f"\nDiscarding {to_discard}")
ENCODINGS = [e for e in ENCODINGS_UNFILTERED if e[0] not in to_discard]

df


,Name,Basic Test,Long Strings,Unordered
0,two_choices,✅,✅,🐖️️️️️️ 0
1,two_choices_wrap,🤷️️️️️️,🤷️️️️️️,🤷️️️️️️
2,two_choices_mark,🤷️️️️️️,🤷️️️️️️,🤷️️️️️️
3,square,✅,✅,✅
4,simplex2d,✅,✅,✅
5,one_child,✅,✅,✅


#### 3. Generate dataset


In [5]:
BIT_STRING_LENGTHS = [100, 300, 500, 700, 900, 1100]
PROBABILITY_OF_ONE = [.1, .2, .3, .4, .5, .6, .7, .8, .9]
AUTOCORRELATION = [.1, .2, .3, .4, .5, .6, .7, .8, .9]
SAMPLES_PER_STRING_TYPE = 10
SAMPLES_PER_STRING_TYPE_AC = 3

# Build generic random data set with random string. 
dataset_generic = [util.gen_bit_string(length, prob_of_one) 
                        for length in BIT_STRING_LENGTHS 
                        for prob_of_one in PROBABILITY_OF_ONE
                        for _ in range(SAMPLES_PER_STRING_TYPE)]

# Inject some edge cases (all zeros, all ones)
dataset_generic += [util.gen_bit_string(length, prob_of_one) 
                        for length in BIT_STRING_LENGTHS 
                        for prob_of_one in [0., 1.]]    


dataset_generic += [util.gen_bit_string_with_autocorrelation(length, ac) 
                        for length in BIT_STRING_LENGTHS 
                        for ac in AUTOCORRELATION
                        for _ in range(SAMPLES_PER_STRING_TYPE_AC)]              

def dataset_for_encoding (encoding_name, bit_to_tree_fun): 
    # Generate random bit strings with various properties
    # Given a bit string, generate a data point
    # [encoding name, length, num nodes]
    def data_point(bit_str):
        tree = bit_to_tree_fun(bit_str)
        str_len = len(bit_str)
        perc_of_ones = bit_str.count('1') / str_len
        autocorrelation = util.autocorrelation(bit_str)
        return [encoding_name, str_len, perc_of_ones, autocorrelation, tree.n_descendants, tree.weight]
    
    return [data_point(bit_str) for bit_str in dataset_generic]

dataset = []
for encoding_name, btt, _ in ENCODINGS:
    dataset += dataset_for_encoding(encoding_name, btt)

df = pd.DataFrame(dataset, columns=['Encoding', 'Bits', 'PercOfOnes', 'Autocorrelation', 'Nodes', 'Weight']) 
df


,Encoding,Bits,PercOfOnes,Autocorrelation,Nodes,Weight
0,two_choices,100,0.080000,0.860000,100,1228
1,two_choices,100,0.190000,0.700000,93,955
2,two_choices,100,0.090000,0.820000,99,1227
3,two_choices,100,0.140000,0.760000,97,1088
4,two_choices,100,0.060000,0.880000,101,1277
...,...,...,...,...,...,...
4279,one_child,1100,0.477273,0.799091,1102,25855
4280,one_child,1100,0.498182,0.786364,1102,29068
4281,one_child,1100,0.493636,0.900000,1102,7430
4282,one_child,1100,0.546364,0.902727,1102,6988


#### 4. Reports

In [6]:
fig = px.box(df, x="Bits", y="Nodes", color="Encoding", 
             title="Bits vs Number Nodes, Entire Dataset")
fig.show()

fig = px.box(df, x="Bits", y="Weight", color="Encoding", 
             title="Weight vs Number Nodes, Entire Dataset")
fig.show()

df_500 = df[df['Bits'] == 500] 
fig = px.scatter(df_500, x="PercOfOnes", y="Nodes", color="Encoding", 
             title="Percentage of Ones vs Num Nodes, 500 Bits")
fig.show()

fig = px.scatter(df_500, x="Autocorrelation", y="Nodes", color="Encoding", 
             title="Autocorrelation vs Num Nodes, 500 Bits")
fig.show()


# Number-Based Evaluation

Some encodings are sensitive to certain numbers, because they are easier to represent in that space. 
Evaluate a contiguous number in a range.

In [13]:
def plot_range(title, center, interval_size, n_samples):
    strings = [(i,util.number_to_bit_string(i)) for i in range(center-interval_size//2,
                                                               center+interval_size//2, 
                                                               interval_size//n_samples)]
    dataset_scatter = []
    for encoding_name, btt, _ in ENCODINGS:
        dataset_scatter += [[encoding_name, i+1, btt(s).n_descendants] for i,s in strings]

    dfs = pd.DataFrame(dataset_scatter, columns=['Encoding', 'Number', 'Nodes']) 
    dfs

    fig = px.line(dfs, x="Number", y="Nodes", color="Encoding", 
                title=f"Number of Nodes in a Range of Numbers -- {title}")
    fig.show()


plot_range("Range", 10002, 20000, 3000)
